In [1]:
import cv2
import os

# Initialize webcam
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Create dataset folder
dataset_path = "dataset"
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

# Ask for user name
user_name = input("Enter your name: ")
user_folder = os.path.join(dataset_path, user_name)

if not os.path.exists(user_folder):
    os.makedirs(user_folder)

print("Capturing images... Press 'q' to quit.")

count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        count += 1
        face_img = gray[y:y+h, x:x+w]
        cv2.imwrite(f"{user_folder}/{count}.jpg", face_img)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Capturing Faces", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q') or count >= 100:
        break

cap.release()
cv2.destroyAllWindows()
print(f"Captured {count} images for {user_name}.")

2025-03-29 15:43:36.563 Python[8042:98318] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Capturing images... Press 'q' to quit.


2025-03-29 15:43:43.656 Python[8042:98318] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-29 15:43:43.656 Python[8042:98318] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Captured 100 images for Neev Gupta.


In [2]:
import cv2
import os
import numpy as np
import pickle

# Ensure OpenCV has the face module
try:
    recognizer = cv2.face.LBPHFaceRecognizer_create()
except AttributeError:
    print("OpenCV is missing the face module. Install opencv-contrib-python:")
    print("pip install opencv-contrib-python")
    exit()

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

dataset_path = "dataset"
labels = []
faces = []
label_dict = {}

current_id = 0

for name in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, name)
    if not os.path.isdir(person_path):
        continue

    label_dict[current_id] = name  # Store label mapping

    for image_name in os.listdir(person_path):
        image_path = os.path.join(person_path, image_name)

        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print(f"Warning: Couldn't read {image_path}, skipping.")
            continue

        faces.append(img)
        labels.append(current_id)

    current_id += 1

# Check if we have enough data to train
if len(faces) < 2:
    print("Not enough data to train. Ensure at least 2 different people have images.")
    exit()

labels = np.array(labels)

print(f"Training on {len(faces)} images from {len(label_dict)} people.")
recognizer.train(faces, labels)

# Save the trained model and label dictionary
recognizer.save("face_model.yml")
with open("labels.pkl", "wb") as f:
    pickle.dump(label_dict, f)

print("Training complete. Model saved as 'face_model.yml'.")

Training on 300 images from 3 people.
Training complete. Model saved as 'face_model.yml'.


In [ ]:
import cv2
import pickle

# Load trained model
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("face_model.yml")

# Load label dictionary
with open("labels.pkl", "rb") as f:
    label_dict = pickle.load(f)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_img = gray[y:y+h, x:x+w]
        
        try:
            label_id, confidence = recognizer.predict(face_img)
            name = label_dict.get(label_id, "Unknown")
            confidence_text = f"{name} ({round(confidence, 2)})"
        except:
            name = "Unknown"
            confidence_text = "Unknown (Error)"

        # Green if confidence < 50, else Red
        color = (0, 255, 0) if confidence < 50 else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, confidence_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()